# Predicting pneumonia from X-ray image: Checkpoint #1

Members: 
- Claire Boyd
- Jack Gibson
- Benjamin Leiva
- Raul Castellanos

In [135]:
# Import modules
import pandas as pd
import matplotlib.pyplot as plt


## Load and prepare data

In this section we'll load the csv files we created using the 'collect_image_files()' function of the load module, and create the necessary variables for analyzing the dataset.

In [136]:
# Load csv files
train = pd.read_csv('data/output/train.csv', usecols=['path', 'channel', 'height', 'width', 'label'])
test  = pd.read_csv('data/output/test.csv', usecols=['path', 'channel', 'height', 'width', 'label'])
val   = pd.read_csv('data/output/val.csv', usecols=['path', 'channel', 'height', 'width', 'label'])

# Create 'type' variable
train['type'] = 'train'
test['type'] =  'test'
val['type'] = 'val'

# Create 'size' variable
train['size'] = train['height'] * train['width']
test['size'] = test['height'] * test['width']
val['size'] = val['height'] * val['width']

# Combine datasets
data = pd.concat([train, test, val], ignore_index=True, axis=0)

# Combined dataset preview
data.head()

,path,channel,height,width,label,type,size
0,NORMAL/NORMAL2-IM-0927-0001.jpeg,1,2234,2359,NORMAL,train,5270006
1,NORMAL/NORMAL2-IM-1056-0001.jpeg,1,1067,1422,NORMAL,train,1517274
2,NORMAL/IM-0427-0001.jpeg,1,1357,1828,NORMAL,train,2480596
3,NORMAL/NORMAL2-IM-1260-0001.jpeg,1,1495,1854,NORMAL,train,2771730
4,NORMAL/IM-0656-0001-0001.jpeg,1,1966,2194,NORMAL,train,4313404


The output from above shows the structure of our data, where each row contains information of an x-ray image. The 'path' column contains the location of the image in the corresponding data type directory (i.e., train, test or val). The 'channel' column 

## Descriptive statistics

The objective of this first

### 1. Label count:

In [137]:
data.groupby(["type", "label"])["type"].count().reset_index(name="count")

,type,label,count
0,test,NORMAL,468
1,test,PNEUMONIA,780
2,train,NORMAL,2682
3,train,PNEUMONIA,7750
4,val,NORMAL,8
5,val,PNEUMONIA,8


Looking at the table above we see that:
- Training data has 2,682 normal labels (26%) and 7,750 pneumonia ones (74%).
- Testing data has 468 normal x-ray labels (38%) and 780 pneumonia ones (62%).
- Validation data has a 50-50 ratio between normal and pneumonia labels. 

### 2. Height, width and size count

From now on we will focus on creating descriptive statistics for the training data, starting with the height and width of our x-ray images.

In [138]:
# Table with descriptive statistics
pd.set_option('display.float_format', lambda x: '%.0f' % x)
train[["height","width", "size"]].describe(percentiles=[0.1, .25, 0.5, 0.75, 0.9])


,height,width,size
count,10432,10432,10432
mean,968,1321,1403850
std,379,355,939101
min,127,384,48768
10%,568,920,531706
25%,688,1056,727888
50%,888,1284,1140640
75%,1188,1552,1841504
90%,1480,1792,2617809
max,2663,2916,7532028


In [139]:
# Height count by bracket
train['height_bracket'] = 'More than 2000 pixels'
train.loc[(train.loc[:, 'height'] < 2000) & (train.loc[:, 'height'] >= 1500), 'height_bracket'] = 'Between 1500-2000 pixels'
train.loc[(train.loc[:, 'height'] < 1500) & (train.loc[:, 'height'] >= 1000), 'height_bracket'] = 'Between 1000-1500 pixels'
train.loc[(train.loc[:, 'height'] < 1000) & (train.loc[:, 'height'] >= 500), 'height_bracket'] = 'Between 500-1000 pixels'
train.loc[(train.loc[:, 'height'] < 500), 'height_bracket'] = 'Less than 500 pixels'

train.groupby(["type", "height_bracket"])["type"].count().reset_index(name="count").sort_values('count', ascending=False)

,type,height_bracket,count
2,train,Between 500-1000 pixels,5694
0,train,Between 1000-1500 pixels,3302
1,train,Between 1500-2000 pixels,802
3,train,Less than 500 pixels,488
4,train,More than 2000 pixels,147


In [140]:
# Width count by bracket
train['width_bracket'] = 'More than 2000 pixels'
train.loc[(train.loc[:, 'width'] < 2000) & (train.loc[:, 'width'] >= 1500), 'width_bracket'] = 'Between 1500-2000 pixels'
train.loc[(train.loc[:, 'width'] < 1500) & (train.loc[:, 'width'] >= 1000), 'width_bracket'] = 'Between 1000-1500 pixels'
train.loc[(train.loc[:, 'width'] < 1000) & (train.loc[:, 'width'] >= 500), 'width_bracket'] = 'Between 500-1000 pixels'
train.loc[(train.loc[:, 'width'] < 500), 'width_bracket'] = 'Less than 500 pixels'

train.groupby(["type", "width_bracket"])["type"].count().reset_index(name="count").sort_values('count', ascending=False)

,type,width_bracket,count
0,train,Between 1000-1500 pixels,5494
1,train,Between 1500-2000 pixels,2698
2,train,Between 500-1000 pixels,1752
4,train,More than 2000 pixels,383
3,train,Less than 500 pixels,106


## Data pre-processing

We decided on two transformations for our training data: CenterCrop and Resize.

The reason for choosing CenterCrop is that removing the borders of x-rays leaves us with the part of the image that gives us information about the presence of pneumonia. This way, we prevent our model from analyzing pixels from, for example, the shoulders or arms regions, and focus on the chest and lungs. Then, we use Resize on the resulting images in order to have homogeneous image sizes in our training data.

Since both of these transformations need height and width parameters to run, we use their observed means in the training data.



In [141]:
# Height mean
height_mean = train['height'].mean()
height_sd = train['height'].std()
print('Mean of height:', int(height_mean))
print('Starndard deviation of height:', int(height_sd))
print('')

# Width mean
width_mean = train['width'].mean()
width_sd = train['width'].std()
print('Mean of width:', int(width_mean))
print('Starndard deviation of width:', int(width_sd))


Mean of height: 968
Starndard deviation of height: 378

Mean of width: 1320
Starndard deviation of width: 355


In [142]:
# Histograms